In [ ]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import collections.abc as abc

# Insertar parametros principales
Insertar los valores de las siguientes variables necesarias para realizar los calculos:  
-  **prestamo**: Valor en Euros del prestamo solicitado al banco
-  **anios**: Numero de años del prestamo, por ejemplo 15
-  **hipoteca_fija**: Insertar:
    - `True` si es una hipoteca con tipo interes fijo (en la variable `tin`)
    - `False` si es una hipoteca con tipo interes variable dependiente de **Euribor** (variable `euribor cambios`)
-  **tin**: Tasa de interes nominal en tanto por 1 (el que suelen indicar los bancos)
    - Ejemplo: Para un valor del 1.29% seria `tin=0.0129`
    - Se utiliza este valor solo si `hipoteca_fija=True`
-  **euribor_inicial**: Valor del Euribor inicial en tanto por 1 al que se le aplican los cambios simulados
    - `euribor_inicial=-0.005` (Valor actual a finales de 2020, es decir, '0.5%)
    - Se utiliza este valor solo si `hipoteca_fija=False`
-  **diferencial**: Valor en tanto por 1 que el banco suma al Euribor para calcular el interes aplicado cada año
    - `diferencial=0.0065` (es decir, se le suma al Euribor un 0.65%)
    - Se utiliza este valor solo si `hipoteca_fija=False`
- **euribor_cambios_dic**: Diccionario con lo que se simula los cambios del Euribor esperados en tanto por 1 **al final de cada año** (los bancos normalmente solo lo revisan 1 vez al año). 
    - Ejemplo: Simular que el Euribor aumenta un 0.1% al año los primeros 5 años, luego es estable 5 años y vuelve a cambiar hasta en el año 11 que aumenta un 0.6% y 0.3 en el 12 y luego en el año 15 baja un 0.1%:
      - `euribor_cambios_dic={1:0.001, 2:0.001, 3:0.001, 4:0.001, 5:0.001, 11:0.006, 12:0.003, 15:-0.001}`
    - Se utiliza este valor solo si `hipoteca_fija=False`
-  **amortizaciones_dic**: Diccionario con el que se simula el dinero en Euros amortizado **al final de cada año**
    - Ejemplo: Simular que los 3 primeros años no se amortiza, luego se empieza a amortizar 5000 euros apartir del cuarto
      -  `amortizaciones_dic={4:5000, 5:5000, 6:5000, 7:5000, 8:5000, 9:5000, 10:5000, 11:5000, 12:5000, 13:5000, 14:5000}`
    - Si no se quieren introducir amortizaciones simplemente poner `amortizaciones_dic=None`
-  **gastos_extra_dic**: Diccionario que simula los gastos adicionales anuales implicados por el banco, como seguros de vida, de hogar etc.
      - Ejemplo: Simular que todos los a ademas de la cuota de la hipoteca hay otros 3000 euros de gastos adicionales en seguros todos los meses
        -  `gastos_extra_dic={'all':3000}`
      - Ejemplo: Simular que los gastos cambian cada año
        - `gastos_extra_dic={1:5000, 2:4000, 3:3000, 4:3000, 5:3000, 6:3000, 7:3000, 8:3000, 9:3000, 10:3000, 11:3000, 12:3000, 13:3000, 14:3000, 15:3000}`
      - Si no se quieren introducir gastos simplemente poner `gastos_extra_dic=None`

In [ ]:
# Parametros rellenados con valores ficticios
prestamo = 300000
anios =  30 
hipoteca_fija = True
tin =  0.0129 # Ej Valor ignorado si hipoteca_fija es False
euribor_inicial = -0.005 # A 20 Dic de 2020 es el valor aproximado
diferencial= 0.0090
euribor_cambios_dic = {1:0.001, 2:0.001, 3:0.001, 4:0.001, 5:0.001, 6:0.001, 7:0.003, 8:0.003, 9:0.003}
amortizaciones_dic= {1:2000, 2:2000, 3:5000, 4:5000, 5:5000, 6:5000, 7:5000, 8:5000, 9:5000, 10:5000, 11:5000, 12:5000}
gastos_extra_dic= {'all':500} # gastos anuales extra totales

# Definicion de funciones necesarias

In [ ]:
def calc_serie_cuota_mensual(tin, n_meses, prestamo, as_array = False):
    i = tin / 12
    r = 1/(1+i)
    cuota = (1-r)*prestamo / (r - r**(n_meses+1)) # = i * prestamo / (1 - (1+i)^(-n_meses))
    if as_array:
        return cuota * np.ones(n_meses)
    else:
        return cuota
    
def crear_serie_amortizaciones(amortizaciones_dic, total_anios):
    amortizaciones_anticipadas = np.zeros(total_anios*12)
    for anio,valor_amortizado in amortizaciones_dic.items():
        amortizaciones_anticipadas[12*anio-1] = valor_amortizado    
    return amortizaciones_anticipadas

def crear_serie_tins(euribor_inicial, diferencial, variaciones_euribor_dic, total_anios):
    tin_mensuales = np.zeros(total_anios*12+1) # Tambien incluyo una posicion 0 inicial que solo capital vivo tiene
    tin_mensuales[1:13] = euribor_inicial + diferencial # Asumo scalar el diferencial por ahora, y solo cambia a final de anio
    for anio in range(1,total_anios):
        tin_mensuales[12*anio+1:12*(anio+1)+1] = tin_mensuales[12*(anio-1)+1:12*(anio)+1]
        if anio in variaciones_euribor_dic.keys():
            tin_mensuales[12*anio+1:12*(anio+1)+1] += variaciones_euribor_dic[anio]
    return tin_mensuales


def crear_serie_gastos_extra(gastos_extra_dic, total_anios):
    if 'all' in gastos_extra_dic.keys():
        gastos_extra = np.ones(total_anios*12) * gastos_extra_dic['all'] / 12
        gastos_extra = np.insert(gastos_extra, 0,0) # Anado un 0 al inicio
    else:
        gastos_extra = np.zeros(total_anios*12+1)
        for anio in range(1,total_anios):
            if anio in gastos_extra_dic.keys():
                gastos_extra[12*(anio-1)+1:12*(anio)+1] = gastos_extra_dic[anio] / 12
    return gastos_extra

def calc_tae(prestamo, cuotas, gastos):
    flujos_caja = np.zeros(len(cuotas)+1) # +1 por el Mes 0
    flujos_caja[0] = -prestamo
    flujos_caja[1:] = cuotas + gastos
    TIR = npf.irr(flujos_caja)
    TAE = (1 + TIR)**12 - 1
    return {"TAE":TAE,"TIR":TIR}

def calc_series_mensuales_deudas_intereses_principal(tin, anios, prestamo,a_anticipadas=None, gastos_adicionales=None):
    n_meses = 12 * anios
    
    # Initialization
    if not isinstance(tin,abc.Sequence):
        tin_mensual = np.ones(n_meses+1)*tin
    else:
        tin_mensual = tin
    capital_vivo = np.zeros(n_meses+1)
    capital_amortizado = np.zeros(n_meses+1)
    intereses = np.zeros(n_meses+1)
    amortizacion = np.zeros(n_meses+1)
    mensualidades=np.zeros(n_meses+1)
    mensualidades_con_amort=np.zeros(n_meses+1) 
    if a_anticipadas is None: # Si no hay amortizaciones anticipadas las inicializo todas a cero
        a_anticipadas = np.zeros(n_meses) # Luego se prependea un elemento siempre, por eso es n_meses long en lugar de n_meses+1
    
    # Mes 0 - Valores que no sean 0s
    capital_vivo[0] = prestamo
    a_anticipadas = np.insert(a_anticipadas, 0,0) # Prepend un 0 en la posicion 0
    
    for mes in range(1,n_meses+1):
        mensualidades[mes] = calc_serie_cuota_mensual(tin=tin_mensual[mes],n_meses=n_meses-mes+1,prestamo=capital_vivo[mes-1]) 
        mensualidades_con_amort[mes]=mensualidades[mes] + a_anticipadas[mes]
        intereses[mes] = capital_vivo[mes-1]*tin_mensual[mes]/12 # Pagas el interes sobre lo que falta por pagar (capital vivo)
        amortizacion[mes] = mensualidades_con_amort[mes] - intereses[mes]
        capital_vivo[mes] = capital_vivo[mes-1] - amortizacion[mes]
        capital_amortizado[mes] = capital_amortizado[mes-1] + amortizacion[mes]
        
    df = pd.DataFrame(data={"mensualidades":mensualidades,"intereses":intereses, "amortizaciones":amortizacion,"capital_vivo":capital_vivo, "capital_amortizado":capital_amortizado, "a_anticipadas":a_anticipadas,"mensualidades_con_amort":mensualidades_con_amort}, index=np.arange(n_meses+1))
    if gastos_adicionales is not None:
        tae_dic = calc_tae(prestamo, df.loc[:,"mensualidades_con_amort"].values[1:], gastos_adicionales[1:])
    else:
        tae_dic=None
    return (df,tae_dic)

# Grafica Principales con TAE y Total Intereses pagados

In [ ]:
if amortizaciones_dic:
    amortizaciones_futuras = crear_serie_amortizaciones(amortizaciones_dic,anios)
else:
    amortizaciones_futuras=None

if gastos_extra_dic:
    gastos_extra = crear_serie_gastos_extra(gastos_extra_dic,anios)
else:
    gastos_extra=None

if hipoteca_fija:
    df,tae_dic = calc_series_mensuales_deudas_intereses_principal(tin=tin, anios=anios, prestamo=prestamo, a_anticipadas=amortizaciones_futuras, gastos_adicionales=gastos_extra)
else:
    tin_variables = crear_serie_tins(euribor_inicial=euribor_inicial,diferencial=diferencial,variaciones_euribor_dic=euribor_cambios_dic,total_anios=anios)
    df,tae_dic = calc_series_mensuales_deudas_intereses_principal(tin=tin_variables, anios=anios, prestamo=prestamo, a_anticipadas=amortizaciones_futuras,  gastos_adicionales=gastos_extra)

# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"intereses"].iloc[1:],
                    mode='lines',
                    name='Interes pagados mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"amortizaciones"].iloc[1:],
                    mode='lines',
                    name='Amortizacion mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"mensualidades"].iloc[1:],
                    mode='lines',
                    name='Pago/cuota mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero de mes")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
if hipoteca_fija:
    title_text=f"TIN fijo {100*tin:.2f}% # Amortizado:{prestamo}€"
else:
    title_text=f"Amortizado:{prestamo}€"

title_text+=f" # Total Intereses:{df.loc[:,'intereses'].iloc[1:].sum():.0f}€"

if tae_dic is not None:
    title_text+=f" # TAE {100*tae_dic['TAE']:.2f}%"


fig.update_layout(title_text=title_text)
fig.show()

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"capital_vivo"].iloc[1:],
                    mode='lines',
                    name='Capital Vivo restante'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"capital_amortizado"].iloc[1:],
                    mode='lines',
                    name='Capital amortizado total'))
fig.add_trace(go.Scatter(x=df.index[1:], y=np.ones(len(df.index[1:]))*prestamo,
                    mode='lines',
                    name='Prestamo Total'))
# Set x-axis title
fig.update_xaxes(title_text="Numero de mes")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
if hipoteca_fija:
    title_text=f"TIN fijo {100*tin:.2f}%"
else:
    title_text=""

title_text+=f" # Total Intereses:{df.loc[:,'intereses'].iloc[1:].sum():.0f}€"

if tae_dic is not None:
    title_text+=f" # TAE {100*tae_dic['TAE']:.2f}%"


fig.update_layout(title_text=title_text)
fig.show()

# Graficas Secundarias (por terminar)

In [ ]:
# Total pagado segun cuando amortizas
q_amort = 15000
anios_amortizar = list(range(1,anios-3))
total_pagado = np.zeros(len(anios_amortizar))
for idx,anio in enumerate(anios_amortizar):
    amortizaciones_dic = {anio:q_amort,anio+1:q_amort,anio+2:q_amort}
    df2 = calc_series_mensuales_deudas_intereses_principal(tin, anios,prestamo,crear_serie_amortizaciones(amortizaciones_dic,anios))
    total_pagado[idx] = df.loc[:,"mensualidades_con_amort"].sum()

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_amortizar, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text=f"Anio amortizar {q_amort} Euros durante 3 anios")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Total pagado segun cuando se amorticen 3 x {q_amort} reduciendo cuotra, no años"
)

fig.show()

In [ ]:
# TIN vs Total pagado
tin_vec = np.arange(0.002,0.0185,0.0005)
total_pagado = np.zeros(len(tin_vec))
for tin_pos, tin_val in enumerate(tin_vec):
    total_pagado[tin_pos] = sum(calc_serie_cuota_mensual(tin=tin_val, n_meses=12*anios, prestamo=prestamo, as_array=True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=100*tin_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Valor del TIN fijo en %")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text="TIN vs Total pagado con 30 anios"
)

fig.show()

In [ ]:
# Anios vs Total pagado
anios_a = np.arange(15,30,1)
total_pagado = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    total_pagado[anios_pos] = sum(calc_serie_cuota_mensual(tin=tin, n_meses=12*anios, prestamo=prestamo, as_array = True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Anios vs Total pagado con tin {100*tin}"
)

fig.show()

In [ ]:
# Anios vs Cuota mensual
anios_a = np.arange(15,30,1)
cuota_mensual = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    cuota_mensual[anios_pos] = calc_serie_cuota_mensual(tin=tin, n_meses=12*anios, prestamo=prestamo, as_array = False)

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=cuota_mensual,
                    mode='lines',
                    name='Cuota Mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Cuota Mensual vs Anios con tin {100*tin}"
)

fig.show()